<div style="display: flex; width: 100%;">
    <div style="flex: 1; padding: 0px;">
        <p>© Albert Palacios Jiménez, 2023</p>
    </div>
    <div style="flex: 1; padding: 0px; text-align: right;">
        <img src="../assets/ieti.png" height="32" alt="Logo de IETI" style="max-height: 32px;">
    </div>
</div>
<hr/>

# Bases de dades desde JAVA

JAVA permet connectar-se a bases de dades relacionals a través de la API JDBC (Java Database Connectivity).

Les característiques de JDBC:

* És fàcil de fer servir

* Pot connectar amb casi tots les bases de dades del mercat

* No té utilitats per traduïr de SQL a objectes Java (ni al revés)

## Funcionament de JDBC

JDBC ens ofereix un object **Connection** genèric que permet connectar a diferents tipus de bases de dades relacionals, a través d'una URL i les dades de connexió.

Cal mantenir una connexió oberta a través de l'objecte *Connection*, i tancar-la amb la funció '.close()' quan ja no la fem servir més.

## Tipus de consultes JDBC

JDBC té dos tipus de consultes bàsiques:

* **executeUpdate** per consultes que no retornen dades en format taula, per exemple: DROP, CREATE, INSERT, UPDATE, ...

* **executeQuery** per consultes que retornen objectes en format de taula (és a dir consultes tipus SELECT)

Quan la consulta és de tipus *taula* i s'ha obtingut amb *executeQuery*, l'objecte que obtenim és un **ResultSet**. Per obtenir informació d'aquest objecte, necessitem saber el tipus de dades que estem agafant segons la columna, per exemple:

* Obtenir el valor enter de la columna etiquetada com a "id": **rs.getInt("id")**

* Obtenir el valor de text de la columna etiquetada com a "name": **rs.getString("name")**

**Nota:** Un *ResultSet* està vinculat a la seva connexió i al Statement que el va generar, i aquests recursos s'han de tancar per evitar fuites de memòria. Un cop tancats, el ResultSet ja no és vàlid. Els exemples copien les dades del *ResultSet* a una nova estructura de dades tipus *List<Map<String, Object>>* que no depèn de la connexió oberta.

## Base de dades SQLite

SQLite és un sistema de gestió de base de dades lleuger, que NO segueix el model client-servidor.

És a dir, guarda la informació en un arxiu enlloc de en un servidor tipus MySQL.

Les característiques de SQLite:

* Soporta usuaris simultanis (però no és molt eficient al fer-ho)

* És "zero-configuration", no cal instal·lar-la i treballa amb un sol arxiu de dades en local

* Permet camps de longitud variable

Per connectar a una base de dades SQLite cal configurar l'arxiu *pom.xml* de maven per afegir la llibreria necessaria:

```xml
<dependencies>
    <dependency>
        <groupId>org.xerial</groupId>
        <artifactId>sqlite-jdbc</artifactId>
        <version>3.36.0.3</version> 
        <!-- Comprova la darrera versió disponible -->
    </dependency>
</dependencies>

```

### Exemple 00

Aquest exemple mostra com fer una connexió amb SQLite, i gestionar dades.

La classe **AppData** és un singleton que gestiona una única connexió amb la base de dades, fent-la accessible des de qualsevol punt del codi.

A més, implementa una funció **query** que enlloc de retornar un **ResultSet** retorna un objecte tipus **List<Map<String, Object>>** que és més senzill de gestionar.

```java
public class Main {
    public static void main(String[] args) {

        // Crear el singleton (això es connecta a la base de dades)
        AppData db = AppData.getInstance();

        System.out.println("\nIniciar les dades de la base de dades:");
        initData();

        System.out.println("\nAnimals de l'espècie 'Gos':");
        llistarFiles("SELECT * FROM animals WHERE especie = 'Gos'");

        System.out.println("\nAnimals de quatre potes:");
        llistarFiles("SELECT * FROM animals WHERE numeropotes = 4");

        // Tancar la connexió amb la base de dades (del singleton)
        db.close();
    }

    public static void initData() {

        // Obtenir un apuntador al singleton de la base de dades
        AppData db = AppData.getInstance();

        // Esborrar la taula 'animals' si existeix
        db.update("DROP TABLE IF EXISTS animals");

        // Crear la taula 'animals'
        db.update("CREATE TABLE IF NOT EXISTS animals (" +
                            "especie TEXT NOT NULL," +
                            "longevitat INTEGER," +
                            "numeropotes INTEGER)");

        // Inserir dades a la taula 'animals'
        db.update("INSERT INTO animals (especie, longevitat, numeropotes) VALUES ('Gos', 14, 4)");
        db.update("INSERT INTO animals (especie, longevitat, numeropotes) VALUES ('Gat', 15, 4)");
        db.update("INSERT INTO animals (especie, longevitat, numeropotes) VALUES ('Elefant', 70, 4)");
        db.update("INSERT INTO animals (especie, longevitat, numeropotes) VALUES ('Tortuga', 100, 4)");
        db.update("INSERT INTO animals (especie, longevitat, numeropotes) VALUES ('Colom', 6, 2)");
    }

    public static void llistarFiles(String sql) {

        // Obtenir un apuntador al singleton de la base de dades
        AppData db = AppData.getInstance();

        List<Map<String, Object>> files = db.query(sql);

        // Llistar el nom i tipus de dades de cada columna (de la fila 0)
        String txt = "Columnes: ";
        Map<String, Object> fila0 = files.get(0);
        for (String key : fila0.keySet()) {
            Object value = fila0.get(key);
            txt += key + " (" + (value != null ? value.getClass().getSimpleName() : "null") + "), ";
        }
        if (files.size() > 0) {
            txt = txt.substring(0, txt.length() -2 );
        }
        System.out.println(txt); 

        // Llistar les files de la query
        System.out.println("Dades:");
        for (Map<String, Object> fila : files) {
            System.out.println(fila.get("especie") + ", " + fila.get("longevitat") + " anys, " + fila.get("numeropotes") + " potes");
        }
    }
}

class AppData {
    private static AppData instance;
    private Connection conn;

    private AppData() {
        connect();
    }

    public static AppData getInstance() {
        if (instance == null) {
            instance = new AppData();
        }
        return instance;
    }

    private void connect() {
        String url = "jdbc:sqlite:dades.db";
        try {
            conn = DriverManager.getConnection(url);
        } catch (SQLException e) {
            System.out.println(e.getMessage());
        }
    }

    public void close() {
        try {
            if (conn != null) conn.close();
        } catch (SQLException e) {
            System.out.println(e.getMessage());
        }
    }

    public void update(String sql) {
        try (Statement stmt = conn.createStatement()) {
            stmt.executeUpdate(sql);
        } catch (SQLException e) {
            System.out.println(e.getMessage());
        }
    }

    // Aquesta funció transforma el ResultSet en un Map<String, Object>
    // per fer l'accés a la informació més genèric
    public List<Map<String, Object>> query(String sql) {
        List<Map<String, Object>> resultList = new ArrayList<>();
        try (Statement stmt = conn.createStatement();
             ResultSet rs = stmt.executeQuery(sql)) {
            ResultSetMetaData metaData = rs.getMetaData();
            int columnCount = metaData.getColumnCount();

            while (rs.next()) {
                Map<String, Object> row = new HashMap<>();
                for (int i = 1; i <= columnCount; i++) {
                    row.put(metaData.getColumnName(i), rs.getObject(i));
                }
                resultList.add(row);
            }
        } catch (SQLException e) {
            System.out.println(e.getMessage());
        }
        return resultList;
    }
}
```

## Base de dades MySQL

Per connectar a una base de dades MySQL cal configurar l'arxiu *pom.xml* de maven per afegir la llibreria necessaria:

```xml
<dependency>
    <groupId>mysql</groupId>
    <artifactId>mysql-connector-java</artifactId>
    <version>8.0.29</version> 
    <!-- Comprova la darrera versió disponible -->
</dependency>
```

Exemple d'obtenir una connexió a MySQL fent servir JDBC:

```java
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.SQLException;

public class MySQLConnectionExample {
    public static void main(String[] args) {
        // Paràmetres de la connexió
        String url = "jdbc:mysql://localhost:3306/nomBaseDades"; 
            // Canvia 'localhost:3306' per la teva adreça i port, 
            // i 'nomBaseDades' pel nom de la teva base de dades
        String user = "usuari"; // Nom d'usuari per a la base de dades
        String password = "contrasenya"; // Contrasenya per a la base de dades

        Connection conn = null;
        try {
            // Carregar el controlador de MySQL
            Class.forName("com.mysql.cj.jdbc.Driver");

            // Establir la connexió amb la base de dades
            conn = DriverManager.getConnection(url, user, password);
            System.out.println("Connexió establerta amb èxit!");

            // Aquí pots afegir el teu codi per a treballar amb la base de dades...

        } catch (ClassNotFoundException e) {
            System.out.println("No s'ha trobat el controlador JDBC.");
            e.printStackTrace();
        } catch (SQLException e) {
            System.out.println("No s'ha pogut establir la connexió.");
            e.printStackTrace();
        } finally {
            // Tancar la connexió
            try {
                if (conn != null && !conn.isClosed()) {
                    conn.close();
                    System.out.println("Connexió tancada.");
                }
            } catch (SQLException e) {
                System.out.println("Error en tancar la connexió.");
                e.printStackTrace();
            }
        }
    }
}
```

### Exemple 01

Aquest exemple mostra com fer una connexió amb MySQL, i gestionar dades.

La classe **AppData** és un singleton que gestiona una única connexió amb la base de dades, fent-la accessible des de qualsevol punt del codi.

A més, implementa una funció **query** que enlloc de retornar un **ResultSet** retorna un objecte tipus **List<Map<String, Object>>** que és més senzill de gestionar.

```java
public class Main {
    public static void main(String[] args) {

        // Crear el singleton (això es connecta a la base de dades)
        AppData db = AppData.getInstance();

        System.out.println("\nLlistar totes les taules:");
        llistarTaules();

        System.out.println("\nLes 10 últimes ciutats de la taula 'city':");
        llistarUltimesCiutats();

        // Tancar la connexió amb la base de dades (del singleton)
        db.close();
    }

    public static void llistarTaules() {
        // Obtenir un apuntador al singleton de la base de dades
        AppData db = AppData.getInstance();

        // Llistar totes les taules
        List<Map<String, Object>> taules = db.query("SHOW TABLES;");

        for (Map<String, Object> taula : taules) {
            // La clau del mapa depèn del nom de la base de dades, per tant, utilitzem el primer valor del mapa.
            System.out.println(taula.values().toArray()[0]);
        }
    }

    public static void llistarUltimesCiutats() {
        // Obtenir un apuntador al singleton de la base de dades
        AppData db = AppData.getInstance();

        // Llistar les 10 últimes ciutats de la taula 'city'
        List<Map<String, Object>> ciutats = db.query("SELECT * FROM city ORDER BY ID DESC LIMIT 10;");

        System.out.println("Dades de les ciutats:");
        for (Map<String, Object> ciutat : ciutats) {
            System.out.println(ciutat.get("Name") + ", " + ciutat.get("CountryCode") + ", " + ciutat.get("District") + ", " + ciutat.get("Population"));
        }
    }
}

package com.project;

import java.sql.*;
import java.util.ArrayList;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

class AppData {
    private static AppData instance;
    private Connection conn;

    private AppData() {
        connect();
    }

    public static AppData getInstance() {
        if (instance == null) {
            instance = new AppData();
        }
        return instance;
    }

    private void connect() {
        // Canvia aquestes variables per les teves credencials reals
        String url = "jdbc:mysql://localhost:3308/world?useSSL=false"; // Utilitza el port 3308 i la base de dades 'world'
        String user = "root"; // El teu usuari de MySQL
        String password = "pwd"; // La teva contrasenya de MySQL

        try {
            // Assegura't que el controlador JDBC de MySQL estigui carregat
            Class.forName("com.mysql.cj.jdbc.Driver");
            conn = DriverManager.getConnection(url, user, password);
        } catch (ClassNotFoundException e) {
            System.out.println("No es pot trobar el controlador JDBC de MySQL.");
            e.printStackTrace();
        } catch (SQLException e) {
            System.out.println("Error connectant a la base de dades MySQL.");
            e.printStackTrace();
        }
    }

    public void close() {
        try {
            if (conn != null) conn.close();
        } catch (SQLException e) {
            System.out.println(e.getMessage());
        }
    }

    public void update(String sql) {
        try (Statement stmt = conn.createStatement()) {
            stmt.executeUpdate(sql);
        } catch (SQLException e) {
            System.out.println(e.getMessage());
        }
    }

    public List<Map<String, Object>> query(String sql) {
        List<Map<String, Object>> resultList = new ArrayList<>();
        try (Statement stmt = conn.createStatement();
             ResultSet rs = stmt.executeQuery(sql)) {
            ResultSetMetaData metaData = rs.getMetaData();
            int columnCount = metaData.getColumnCount();

            while (rs.next()) {
                Map<String, Object> row = new HashMap<>();
                for (int i = 1; i <= columnCount; i++) {
                    row.put(metaData.getColumnName(i), rs.getObject(i));
                }
                resultList.add(row);
            }
        } catch (SQLException e) {
            System.out.println(e.getMessage());
        }
        return resultList;
    }
}
```